In [ ]:
import gensim
from gensim.models import doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from smart_open import smart_open
import logging
import os
import networkx as nx
from networkx.drawing.nx_pydot import write_dot, read_dot
import json
import numpy as np
import pandas as pd
import re
from wikipedia2vec import Wikipedia2Vec
import time

In [ ]:
# read from random walks file to python stream
# one walk at a time
class Documents:
    """
    w.r.t docs.txt & doc2.txt
    """
    def __init__(self, path):
        self.path = path
        self.tags = []
    def __iter__(self):
        count = 0
        for line in smart_open(self.path, 'r'):
            full_name = line.split(':::')[0].strip()
            random_walks = line.split(':::')[1].strip().split('$$$')
            words = []
            for walk in random_walks:
                for w in walk.split(','):
                    words.append(w)
            l = int(len(words) / 2) 
            tag = full_name
            yield TaggedDocument(words[:l], [tag])
            self.tags.append(tag)
    def iter(self):
        for doc in documents:
            continue
    def find_words_by_tag(self, tag):
        if len(self.tags) == 0:
            self.iter()
        return self.dictionary[tag]

In [ ]:
# read from nature language file to python stream
# one sentence at a time
class NLDocuments:
    """w.r.t. natural_info.txt"""
    def __init__(self, path):
        self.path = path
        self.tags = []
    def __iter__(self):
        count = 0
        for line in smart_open(self.path, 'r'):
            full_name = line.split(':::')[0].strip()
            infos = line.split(':::')[1].strip().split('+++')
#             del infos[1]
            words = []
            for info in infos:
                for w in NLDocuments._split(info):
                    w = NLDocuments.filter_alphabetic(w)
                    if len(w) > 0 and not w.isdigit():
                        words.append(w)
            tag = full_name
            yield TaggedDocument(words, [tag])
            count += 1
            self.count = count
            self.tags.append(tag)
    @staticmethod
    def filter_alphabetic(input):
        regex = re.compile('[^a-zA-Z]')
        return regex.sub('', input)
    @staticmethod
    def _split(input):
        tokens = []
        for t in input.split():
            for t2 in t.split('.'):
                t_u = ''
                for _s_ in t2:
                    if isinstance(_s_, str):
                        t_u += _s_ if _s_.islower() else '_' + _s_.lower()
                l = t_u.split('_')
                for r in l:
                    if r != '' and not r.isdigit():
                        tokens.append(r)
        return tokens

In [ ]:
# common utils
class Utils:
    def __init__(self, path):
        self.path = path
        self.data = []
    def read_data(self):
        data= []
        with open(self.path, 'r') as f:
            for line in f.readlines():
                for word in line.split():
                    data.append(word)
        self.data = data
        return data
    def count_unique(self):
        self.read_data()
        s = set(self.data)
        return len(s)
    def get_unique_words(self):
        self.read_data()
        s = set(self.data)
        return s

In [ ]:
# word2vec model from gensim
# add visualization and callback
class Word2vec_:
    def __init__(self, input, output, log):
        self.input = input
        self.output = output
        self.log = log
    def train(self):
        """last train indicates 25 iters reach loss min"""
#         logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        sentences = Sentences(input)
        model = Word2Vec(sentences, sg=1, negative=20, size=228, window=5, min_count=32, workers=16, iter=30, compute_loss=True,
            callbacks=[self.callback(log)])
        model.save(output)
    @staticmethod
    def write_tsv(model, meta, emb):
        model = Word2Vec.load(model)
        with open(meta, 'w') as f1:
            with open(emb, 'w') as f2:
                for i in model.wv.vocab:
                    f1.write(str(i) + '\n')
                    vecs = model.wv[i]
                    for j in range(len(vecs)-1):
                        v = str(vecs[j])
                        f2.write(v + '\t')
                    v = str(vecs[len(vecs)-1])
                    f2.write(v)
                    f2.write('\n')
    class callback(CallbackAny2Vec):
        '''print loss every epoch'''
        def __init__(self, log_path):
            self.path = log_path
            self.epoch = 0

        def on_epoch_end(self, model):
            with open(self.path, 'a') as f:
                loss = model.running_training_loss
                print('Loss after epoch {}: {}'.format(self.epoch, loss))
                f.write('Loss after epoch {}: {}'.format(self.epoch, loss))
                f.write('\n')
                self.epoch += 1

In [ ]:
# doc2vec model from gensim
# add visualization and callback
class Doc2vec_:
    def __init__(self, input, output, log):
        self.input = input
        self.output = output
        self.log = log
        
    def train(self, size):
        print('time: {}'.format(time.localtime()))
        docs = Documents(self.input)
        model = Doc2Vec(docs, dm_mean=0, min_count=8, window=5, workers=16, iter=20, compute_loss=True, negative=20, size=size, callbacks=[self.callback(log)])
        model.save(output)
        return docs
        
    @staticmethod
    def write_wv_tsv(model, meta, emb):
        with open(meta, 'w') as f1:
            with open(emb, 'w') as f2:
                for i in sorted(model.wv.vocab):
                    f1.write(str(i) + '\n')
                    vecs = model.wv[i]
                    for j in range(len(vecs)-1):
                        v = str(vecs[j])
                        f2.write(v + '\t')
                    v = str(vecs[len(vecs)-1])
                    f2.write(v)
                    f2.write('\n')
    @staticmethod
    def write_docvecs_tsv(model, documents, meta, emb):
        with open(meta, 'w') as f1:
            with open(emb, 'w') as f2:
                for tag in documents.tags:
                    f1.write(tag + '\n')
                    vecs = model.docvecs[tag]
                    for j in range(len(vecs)-1):
                        v = str(vecs[j])
                        f2.write(v + '\t')
                    v = str(vecs[len(vecs)-1])
                    f2.write(v)
                    f2.write('\n')
    @staticmethod
    def get_vector_by_tag(model, tag):
        return model.docvecs[tag]
    
    class callback(CallbackAny2Vec):
        '''print loss every epoch'''
        def __init__(self, log_path):
            self.epoch = 0
            self.path = log_path

        def on_epoch_end(self, model):
            print('epoch: {}'.format(self.epoch))
            print('time: {}'.format(time.localtime()))
            with open(self.path, 'a') as f:
                f.write('epoch: {}'.format(self.epoch))
                f.write('\n')
            self.epoch += 1

In [ ]:
# word2vec model from gensim(to train nl-embedding)
# add visualization and callback
class Doc2vec_NL:
    def __init__(self, input, output, log):
        self.input = input
        self.output = output
        self.log = log
        
    def train(self, size):
        print('time: {}'.format(time.localtime()))
        docs = NLDocuments(self.input)
        model = Doc2Vec(docs, dm_mean=0, min_count=8, window=5, workers=16, iter=60, compute_loss=True, negative=20, size=size, callbacks=[self.callback(log)])
        model.save(output)
        
    @staticmethod
    def write_wv_tsv(model, meta, emb):
        with open(meta, 'w') as f1:
            with open(emb, 'w') as f2:
                for i in sorted(model.wv.vocab):
                    f1.write(str(i) + '\n')
                    vecs = model.wv[i]
                    for j in range(len(vecs)-1):
                        v = str(vecs[j])
                        f2.write(v + '\t')
                    v = str(vecs[len(vecs)-1])
                    f2.write(v)
                    f2.write('\n')
    @staticmethod
    def write_docvecs_tsv(model, documents, meta, emb):
        with open(meta, 'w') as f1:
            with open(emb, 'w') as f2:
                for tag in documents.tags:
                    f1.write(tag + '\n')
                    vecs = model.docvecs[tag]
                    for j in range(len(vecs)-1):
                        v = str(vecs[j])
                        f2.write(v + '\t')
                    v = str(vecs[len(vecs)-1])
                    f2.write(v)
                    f2.write('\n')
    @staticmethod
    def get_vector_by_tag(model, tag):
        return model.docvecs[tag]
    
    class callback(CallbackAny2Vec):
        '''print loss every epoch'''
        def __init__(self, log_path):
            self.epoch = 0
            self.path = log_path

        def on_epoch_end(self, model):
            print('epoch: {}'.format(self.epoch))
            print('time: {}'.format(time.localtime()))
            with open(self.path, 'a') as f:
                f.write('epoch: {}'.format(self.epoch))
                f.write('\n')
            self.epoch += 1

# graph2vec method

In [ ]:
"""
for training: run shell in command line
"""

In [ ]:
"""csv 2 tsv"""
def csv2tsv(input, meta_tsv, emb_tsv):
    """translate output features in json file into tsv file, for tensorboard """
    data = pd.read_csv(input, sep=',')
    embeddings = data.iloc[:,1:].values
    meta = data.iloc[:,0].values
    with open(meta_tsv, 'w') as f1:
        for index in meta:
            f1.write(str(index))
            f1.write('\n')
    with open(emb_tsv, 'w') as f2:
        for line in embeddings:
            for i in range(len(line)-1):
                v = str(line[i])
                f2.write(v + '\t')
            f2.write(str(line[len(line)-1]) + '\n')
input = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/graph2vec/result.csv'
meta = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/graph2vec/metadata.tsv'
emb = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/graph2vec/embeddings.tsv'
csv2tsv(input, meta, emb)

In [ ]:
"""load id name dictionary"""
id_dictionary = {}
reverse_id_dictionary = {}
with open('/home/qwe/zfy_lab/SoC/data/files/IDs.txt', 'r') as f:
    for line in f.readlines():
        full = line.split(':::')[0]
        id = int(line.split(':::')[1])
        id_dictionary.setdefault(full, None)
        id_dictionary[full] = id
        reverse_id_dictionary.setdefault(id, None)
        reverse_id_dictionary[id] = full

In [ ]:
len(reverse_id_dictionary)

In [ ]:
"""get meta2 (real name)"""
with open('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/graph2vec/metadata.tsv', 'r') as f1:
    with open('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/graph2vec/metadata2.tsv', 'w') as f2:
        for line in f1.readlines():
            id = int(line.strip())
            real_name = reverse_id_dictionary[id]
            f2.write(real_name)
            f2.write('\n')
    

In [ ]:
"""get meta3 (simple name)"""
f1 = open('/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/graph2vec/metadata2.tsv', 'r')
f2 = open('/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/graph2vec/metadata3.tsv', 'w')
for line in f1.readlines():
    real_name = line.strip()
    p = re.compile('\(.*\)')
    simple_name = re.sub(p, '', real_name)
    simple_name = simple_name.split('.')[-1]
    f2.write(simple_name + '\n')

# train jinple embedding

In [ ]:
input = '/home/qwe/disk1/data_SoC/files_20w/random_walks.txt'
output = '/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/jimple2vec/times4/doc2vec.model'
log = '/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/jimple2vec/times4/log.txt'
doc2vec = Doc2vec_(input, output, log)

In [ ]:
doc2vec.train(228)

In [ ]:
model = Doc2Vec.load('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/jimple2vec/more_epochs/doc2vec.model')

In [ ]:
documents = Documents('/home/qwe/disk1/data_SoC/files/random_walks.txt')
for doc in documents:
    pass

In [ ]:
tsv1 = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/jimple2vec/more_epochs/metadata.tsv'
tsv2 = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/jimple2vec/more_epochs/embeddings.tsv'
Doc2vec_.write_docvecs_tsv(model, documents, tsv1, tsv2)

In [ ]:
with open ('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/jimple2vec/more_epochs/metadata.tsv', 'r')as f1:
    with open('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/jimple2vec/more_epochs/metadata2.tsv', 'w') as f2:
        for line in f1.readlines():
            name = line.strip()
            simple = re.sub('\(.*\)', '', name)
            simple = simple.split('.')[-1]
            f2.write(simple + '\n')

In [ ]:
online_model = Doc2Vec.load('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/jimple2vec/doc2vec.model')

In [ ]:
len(documents.tags)

# train natural language embedding

In [ ]:
input = '/home/qwe/disk1/data/method-name-predicttion/RQ2/no-trans/natural_info.txt'
output = '/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/nl2vec/no_trans/doc2vec_nl.model'
log = '/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/nl2vec/no_trans/log.txt'
nl_doc2vec = Doc2vec_NL(input, output, log)

In [ ]:
nl_doc2vec.train(size=372)

In [ ]:
nldoc = NLDocuments('/home/qwe/disk1/data_SoC/files/natural_info.txt')
for nl in nldoc:
    continue

In [ ]:
model = Doc2Vec.load('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/nl2vec/more_epochs/doc2vec_nl.model')
tsv1 = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/nl2vec/more_epochs/metadata.tsv'
tsv2 = '/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/nl2vec/more_epochs/embeddings.tsv'
Doc2vec_NL.write_docvecs_tsv(model, nldoc, tsv1, tsv2)

In [ ]:
with open ('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/nl2vec/more_epochs/metadata.tsv', 'r')as f1:
    with open('/home/qwe/disk1/models/unsupervised-learning-results/result_08.06/nl2vec/more_epochs/metadata2.tsv', 'w') as f2:
        for line in f1.readlines():
            name = line.strip()
            simple = re.sub('\(.*\)', '', name)
            simple = simple.split('.')[-1]
            f2.write(simple + '\n')

# embedding combine (discard sc-embedding)

In [ ]:
"""
now we have 3 embeddings, each are:
1. id : sc-embedding (discarded)
2. full : jimple-embedding
3. full : nl-embedding
we chose to use id as common identifier
"""

In [ ]:
# get id - full name dictionary
id_dictionary = {}
reverse_id_dictionary = {}
with open('/home/qwe/disk1/data_SoC/files_20w/IDs.txt', 'r') as f:
    for line in f.readlines():
        full = line.split(':::')[0]
        id = int(line.split(':::')[1])
        id_dictionary.setdefault(full, None)
        id_dictionary[full] = id
        reverse_id_dictionary.setdefault(id, None)
        reverse_id_dictionary[id] = full

In [ ]:
len(id_dictionary), len(reverse_id_dictionary)

In [ ]:
# # get sc-embedding dictionary and normalize
# sc_dictionary = {}
# data = pd.read_csv('/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/graph2vec/result.csv', sep=',')
# embeddings = data.iloc[:,1:].values
# meta = data.iloc[:,0].values
# assert len(meta) == len(embeddings)
# for i in range(len(meta)):
#     sc_dictionary.setdefault(meta[i], None)
#     v = embeddings[i]
#     sums = v.sum(axis=0)
#     v2 = v / sums
#     sc_dictionary[meta[i]] = v2
    

In [ ]:
# get jimple embedding dictionary
model = Doc2Vec.load('/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/jimple2vec/times6/doc2vec.model')
documents = Documents('/home/qwe/disk1/data_SoC/files_20w/random_walks.txt')
for doc in documents:
    continue

In [ ]:
jimple_dictionary = {}
for tag in documents.tags:
    id = id_dictionary[tag]
    emb = model.docvecs[tag]
    jimple_dictionary.setdefault(id, None)
    jimple_dictionary[id] = emb

In [ ]:
len(jimple_dictionary)

In [ ]:
# get nl embedding dictionary
nldoc = NLDocuments('/home/qwe/disk1/data_SoC/files_20w/natural_info.txt')
for nl in nldoc:
    continue
model = Doc2Vec.load('/home/qwe/disk1/models/unsupervised-learning-results/result_7.31/nl2vec/doc2vec_nl.model')

In [ ]:
nl_dictionary = {}
for tag in nldoc.tags:
    id = id_dictionary[tag]
    emb = model.docvecs[tag]
    nl_dictionary.setdefault(id, None)
    nl_dictionary[id] = emb

In [ ]:
# test dictionaries
assert len(jimple_dictionary) == len(nl_dictionary)
assert len(jimple_dictionary[72]) + len(nl_dictionary[72]) == 600
assert jimple_dictionary.keys() == nl_dictionary.keys()

# write embeddings to file

In [ ]:
valid_id_list = []
with open('/home/qwe/disk1/data/method-name-predicttion/training-data_08.02/train1/valid_id.txt', 'r') as f:
    for line in f.readlines():
        valid_id_list.append(int(line.strip()))
len(valid_id_list)

In [ ]:
# write to file
target_path = '/home/qwe/disk1/data/method-name-predicttion/training-data_08.02/train1/train_times1.csv'
with open(target_path, 'w') as f:
    for id in valid_id_list:
        v1 = sc_dictionary[id]
        v2 = jimple_dictionary[id]
        v3 = nl_dictionary[id]
        v = np.concatenate((v1,v2,v3))
        assert len(v) == 900
        v = [str(r) for r in v]
        assert len(v) == 900
        f.write(','.join(v))
        f.write('\n')